In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import math

# --- Helper function for angle comparison ---
def is_angle_in_range(angle, start_angle, end_angle):
    """
    Checks if 'angle' is within the range [start_angle, end_angle],
    handling wrapping around the -pi/pi boundary.
    All angles are assumed to be in radians, typically from atan2 (-pi to pi).
    """
    # Normalize all angles to be in [0, 2*pi) for easier comparison
    angle = (angle + 2 * math.pi) % (2 * math.pi)
    start_angle = (start_angle + 2 * math.pi) % (2 * math.pi)
    end_angle = (end_angle + 2 * math.pi) % (2 * math.pi)

    if start_angle <= end_angle:
        # Normal range (e.g., 30 to 120 degrees)
        return start_angle <= angle <= end_angle
    else:
        # Range wraps around 0 (or 2*pi), e.g., 300 to 60 degrees
        return angle >= start_angle or angle <= end_angle

# --- 1. Room and Sensor Model (with Obstacle Blocking Logic) ---

class Room:
    """
    Represents the 2D room with its dimensions and obstacles.
    It provides methods to check if points are inside the room or obstructed.
    """
    def __init__(self, width, height, obstacles=None):
        self.width = width
        self.height = height
        # Obstacles defined as (x, y, width, height) of rectangular blocks
        self.obstacles = obstacles if obstacles is not None else []
        self.grid_resolution = 0.5  # Meters per grid cell for discretizing the room
        self.grid_x = int(self.width / self.grid_resolution) # Number of grid cells in X
        self.grid_y = int(self.height / self.grid_resolution) # Number of grid cells in Y
        self.coverage_grid = np.zeros((self.grid_x, self.grid_y)) # Grid to track coverage

    def is_inside(self, x, y):
        """Checks if a point is within the room boundaries."""
        return 0 <= x <= self.width and 0 <= y <= self.height

    def is_obstructed_at_point(self, x, y):
        """Checks if a single point is inside any obstacle."""
        for obs_x, obs_y, obs_w, obs_h in self.obstacles:
            # Check if the point (x, y) falls within the current obstacle's bounds
            if obs_x <= x <= obs_x + obs_w and obs_y <= y <= obs_h:
                return True
        return False

    def is_line_obstructed(self, p1, p2, step_size=0.1):
        """
        Checks if the line segment between p1 and p2 is obstructed by any obstacle.
        Uses a basic ray-casting approach by stepping along the line.
        p1: (x1, y1) - sensor position
        p2: (x2, y2) - point in room grid
        step_size: How finely to check along the line. Smaller is more accurate but slower.
        """
        dist = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
        if dist == 0:
            # If p1 and p2 are the same point, check if that point is obstructed
            return self.is_obstructed_at_point(p1[0], p1[1])

        # Determine the number of steps to take along the line
        num_steps = int(dist / step_size) + 1
        for i in range(num_steps):
            t = i / num_steps # Interpolation parameter from 0 to 1
            # Interpolate a point along the line segment
            px = p1[0] + t * (p2[0] - p1[0])
            py = p1[1] + t * (p2[1] - p1[1])

            # Check if this interpolated point is inside any obstacle
            if self.is_obstructed_at_point(px, py):
                return True # Obstruction found
        return False # No obstruction found along the line


class Sensor:
    """
    Represents a PIR sensor with a position, detection range, orientation, and field of view.
    Calculates which grid points it can cover, considering room boundaries, obstacles,
    and its directional field of view.
    """
    def __init__(self, x, y, detection_range, orientation_deg, fov_deg):
        self.x = x
        self.y = y
        self.detection_range = detection_range
        self.orientation = math.radians(orientation_deg) # Convert orientation to radians
        self.fov = math.radians(fov_deg) # Convert FOV to radians

    def get_covered_points(self, room):
        """
        Returns a list of (grid_x, grid_y) tuples for cells covered by this sensor,
        considering detection range, obstacles blocking line of sight, and FOV.
        """
        covered_points = []
        sensor_pos = (self.x, self.y)

        # First, check if the sensor itself is placed inside an obstacle.
        # If so, it cannot cover anything.
        if room.is_obstructed_at_point(self.x, self.y):
            return []

        # Define the angular range for the FOV
        half_fov = self.fov / 2
        min_angle = self.orientation - half_fov
        max_angle = self.orientation + half_fov

        for gx in range(room.grid_x):
            for gy in range(room.grid_y):
                # Calculate the real-world coordinates of the center of the grid cell
                px = gx * room.grid_resolution + room.grid_resolution / 2
                py = gy * room.grid_resolution + room.grid_resolution / 2
                grid_point_pos = (px, py)

                # 1. Check if the grid point is within the room boundaries
                if not room.is_inside(px, py):
                    continue

                # 2. Check if the grid point itself is an obstacle.
                # Obstacles cannot be "covered" as they are solid objects.
                if room.is_obstructed_at_point(px, py):
                    continue

                # 3. Check distance: Is the grid point within the sensor's detection range?
                distance = np.sqrt((self.x - px)**2 + (self.y - py)**2)
                if distance > self.detection_range:
                    continue

                # 4. Check if the angle to the grid point is within the sensor's FOV
                angle_to_point = math.atan2(py - self.y, px - self.x)
                if not is_angle_in_range(angle_to_point, min_angle, max_angle):
                    continue # Not within FOV

                # 5. Check for line-of-sight obstruction between the sensor and the grid point.
                # This is crucial for identifying "dead areas" behind obstacles.
                if not room.is_line_obstructed(sensor_pos, grid_point_pos):
                    covered_points.append((gx, gy)) # If all checks pass, the cell is covered
        return covered_points

# --- 2. Objective Function ---

def calculate_coverage(sensor_params, room, num_sensors, detection_range, fov_deg):
    """
    Calculates the total covered area for a given set of sensor parameters (x, y, orientation),
    now incorporating obstacle blocking and directional FOV.
    sensor_params: flat array [x1, y1, orientation1, x2, y2, orientation2, ...]
    """
    current_coverage_grid = np.zeros_like(room.coverage_grid) # A grid to mark covered cells
    sensors = []
    # Create Sensor objects from the flattened sensor_params array
    for i in range(num_sensors):
        sx = sensor_params[3 * i]
        sy = sensor_params[3 * i + 1]
        s_orientation_rad = sensor_params[3 * i + 2] # Orientation is now in radians from optimization
        # Convert back to degrees for the Sensor constructor, or just pass radians directly
        sensors.append(Sensor(sx, sy, detection_range, math.degrees(s_orientation_rad), fov_deg))

    total_covered_cells = 0
    # Iterate through each sensor and mark the cells it covers
    for sensor in sensors:
        covered_points = sensor.get_covered_points(room)
        for gx, gy in covered_points:
            # Only count cells that haven't been covered yet to avoid double-counting
            if current_coverage_grid[gx, gy] == 0:
                current_coverage_grid[gx, gy] = 1 # Mark as covered
                total_covered_cells += 1

    return total_covered_cells

def objective_function(sensor_params, room, num_sensors, detection_range, fov_deg):
    """
    Objective function for the optimization. We want to maximize coverage,
    so we minimize the negative of the calculated coverage.
    """
    coverage = calculate_coverage(sensor_params, room, num_sensors, detection_range, fov_deg)
    return -coverage # Minimize negative coverage (equivalent to maximizing coverage)

# --- 3. Optimization Setup and Execution ---

# Room parameters
room_width = 10
room_height = 8

# UPDATED OBSTACLE DEFINITIONS: Two smaller cabinets
room_obstacles = [
    (2.5, 0.5, 0.5, 2.0),  # Smaller cabinet 1 (x, y, width, height)
    (6.5, 3.0, 0.5, 2.0)   # Smaller cabinet 2 (x, y, width, height)
]
room = Room(room_width, room_height, room_obstacles)

# Sensor parameters
num_sensors = 5
sensor_detection_range = 5.0 # Detection range of each sensor
pir_fov_degrees = 100 # Field of View for the PIR sensor

# Define bounds for sensor placement (x, y) and orientation (radians)
bounds = []
for _ in range(num_sensors):
    bounds.append((0, room_width))   # x-coordinate bounds
    bounds.append((0, room_height))  # y-coordinate bounds
    bounds.append((0, 2 * np.pi))    # orientation bounds (0 to 2*pi radians)

print("Starting optimization...")
# Run the differential evolution optimization algorithm
result = differential_evolution(
    func=objective_function,
    bounds=bounds,
    args=(room, num_sensors, sensor_detection_range, pir_fov_degrees),
    strategy='best1bin', # Optimization strategy
    maxiter=300,         # Increased iterations for better convergence
    popsize=30,          # Increased population size
    tol=0.01,            # Tolerance for convergence
    disp=True            # Display optimization progress
)

optimal_sensor_params = result.x
optimal_coverage = -result.fun # Convert the minimized negative coverage back to positive coverage

print(f"\nOptimal Sensor Parameters (x, y, orientation_rad):\n{optimal_sensor_params.reshape(num_sensors, 3)}")
print(f"Optimal Covered Cells: {optimal_coverage}")
print(f"Total possible cells in grid: {room.grid_x * room.grid_y}")
# Calculate the total non-obstacle cells for a more accurate coverage percentage
total_non_obstacle_cells = 0
for gx in range(room.grid_x):
    for gy in range(room.grid_y):
        px = gx * room.grid_resolution + room.grid_resolution / 2
        py = gy * room.grid_resolution + room.grid_resolution / 2
        if not room.is_obstructed_at_point(px, py):
            total_non_obstacle_cells += 1

print(f"Total non-obstacle cells: {total_non_obstacle_cells}")
print(f"Coverage Percentage (of non-obstacle cells): {100 * optimal_coverage / total_non_obstacle_cells:.2f}%")


# --- 4. Visualization (using Matplotlib) ---

def plot_room_and_coverage(room, sensors_data, detection_range, fov_deg):
    """
    Plots the room, obstacles, sensor positions, and the calculated coverage area.
    sensors_data: Array of [x, y, orientation_rad] for each sensor
    """
    # Adjust figure size based on room dimensions for better aspect ratio
    fig, ax = plt.subplots(figsize=(room.width * 0.8, room.height * 0.8))

    # Plot room boundaries
    ax.add_patch(plt.Rectangle((0, 0), room.width, room.height, fill=False, edgecolor='black', linewidth=2))

    # Plot obstacles
    for obs_x, obs_y, obs_w, obs_h in room.obstacles:
        ax.add_patch(plt.Rectangle((obs_x, obs_y), obs_w, obs_h, facecolor='gray', edgecolor='black', alpha=0.8))

    # Create Sensor objects for plotting (re-initialize with optimal parameters)
    current_sensors = []
    for i in range(len(sensors_data)):
        sx, sy, s_orientation_rad = sensors_data[i]
        current_sensors.append(Sensor(sx, sy, detection_range, math.degrees(s_orientation_rad), fov_deg))

    # Calculate the final coverage map for plotting
    coverage_map = np.zeros((room.grid_x, room.grid_y))
    for sensor in current_sensors:
        covered_points = sensor.get_covered_points(room)
        for gx, gy in covered_points:
            coverage_map[gx, gy] = 1 # Mark as covered

    # Create meshgrid for plotting the cells
    x_coords = np.arange(0, room.width, room.grid_resolution)
    y_coords = np.arange(0, room.height, room.grid_resolution)

    # Plot covered cells using pcolormesh for a clear grid representation
    # Transpose coverage_map for correct orientation with pcolormesh
    plt.pcolormesh(x_coords, y_coords, coverage_map.T, cmap='Blues', alpha=0.5, shading='auto')


    # Plot sensors and their theoretical detection cones
    for sensor in current_sensors:
        ax.plot(sensor.x, sensor.y, 'ro', markersize=8, label='Sensor Location') # Red circle for sensor

        # Draw the FOV cone
        # Generate points for the arc
        arc_angles = np.linspace(sensor.orientation - sensor.fov / 2,
                                 sensor.orientation + sensor.fov / 2,
                                 50) # 50 points for a smooth arc
        arc_x = sensor.x + sensor.detection_range * np.cos(arc_angles)
        arc_y = sensor.y + sensor.detection_range * np.sin(arc_angles)

        # Combine with sensor position to form a filled polygon (cone)
        cone_x = [sensor.x] + list(arc_x) + [sensor.x]
        cone_y = [sensor.y] + list(arc_y) + [sensor.y]
        ax.add_patch(plt.Polygon(list(zip(cone_x, cone_y)), color='blue', alpha=0.1, linewidth=0)) # Filled cone

        # Draw the cone boundaries
        ax.plot([sensor.x, sensor.x + sensor.detection_range * np.cos(sensor.orientation - sensor.fov / 2)],
                [sensor.y, sensor.y + sensor.detection_range * np.sin(sensor.orientation - sensor.fov / 2)],
                'b--', alpha=0.6, linewidth=1)
        ax.plot([sensor.x, sensor.x + sensor.detection_range * np.cos(sensor.orientation + sensor.fov / 2)],
                [sensor.y, sensor.y + sensor.detection_range * np.sin(sensor.orientation + sensor.fov / 2)],
                'b--', alpha=0.6, linewidth=1)
        ax.plot(arc_x, arc_y, 'b--', alpha=0.6, linewidth=1) # Arc boundary


    # Set plot limits and labels
    ax.set_xlim(0, room.width)
    ax.set_ylim(0, room.height)
    ax.set_aspect('equal', adjustable='box') # Maintain aspect ratio
    ax.set_xlabel('X (meters)')
    ax.set_ylabel('Y (meters)')
    ax.set_title('Optimal PIR Sensor Placement with Obstacle Blocking and Dead Areas')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

# Prepare the final sensor data for plotting based on the optimization result
# Reshape optimal_sensor_params to (num_sensors, 3) for easier access
final_sensor_data = optimal_sensor_params.reshape(num_sensors, 3)

# Generate the plot
plot_room_and_coverage(room, final_sensor_data, sensor_detection_range, pir_fov_degrees)

Starting optimization...
differential_evolution step 1: f(x)= -264.0
differential_evolution step 2: f(x)= -274.0
differential_evolution step 3: f(x)= -274.0
differential_evolution step 4: f(x)= -274.0
differential_evolution step 5: f(x)= -277.0
differential_evolution step 6: f(x)= -277.0
differential_evolution step 7: f(x)= -277.0
differential_evolution step 8: f(x)= -277.0
differential_evolution step 9: f(x)= -281.0
differential_evolution step 10: f(x)= -281.0
differential_evolution step 11: f(x)= -281.0
differential_evolution step 12: f(x)= -281.0
differential_evolution step 13: f(x)= -281.0
differential_evolution step 14: f(x)= -281.0
differential_evolution step 15: f(x)= -281.0
differential_evolution step 16: f(x)= -288.0
differential_evolution step 17: f(x)= -288.0
differential_evolution step 18: f(x)= -288.0
differential_evolution step 19: f(x)= -288.0
differential_evolution step 20: f(x)= -288.0
differential_evolution step 21: f(x)= -288.0
differential_evolution step 22: f(x)= -